In [37]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [38]:
root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

In [39]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)

In [40]:
x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

In [41]:
#convert y to 0 and 1
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [42]:
#normalize x
x_train = MinMaxScaler().fit_transform(x_train)
x_test = MinMaxScaler().fit_transform(x_test)

In [43]:
#expand dimension
x_train = np.expand_dims(x_train, axis=1)
x_test = np.expand_dims(x_test, axis=1)

In [44]:
#check shapes
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(3601, 1, 500) (3601,) (1320, 1, 500) (1320,)


In [45]:
#convert to torch tensors
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

In [46]:
#convert to torch dataset
train = TensorDataset(x_train, y_train)
test = TensorDataset(x_test, y_test)
#convert to dataloader
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=False)

In [47]:
#check shapes
for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([64, 1, 500]) torch.Size([64])


In [56]:
#1D CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.sequential = nn.Sequential(
            nn.Conv1d(1, 64, 3),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 64, 3),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 64, 3),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.AvgPool1d(3),
            #flatten
            nn.Flatten(),
            nn.LazyLinear(out_features=2)
        )
    def forward(self, x):
        x = self.sequential(x)
        return x

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

In [59]:
#feed a sample to the model
for x, y in train_loader:
    x = x.to(device)
    print(model(x).shape)
    break

torch.Size([64, 2])


In [60]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [61]:
#training
n_epochs = 10
for epoch in range(n_epochs):
    train_loss = 0.0
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
    train_loss = train_loss / len(train_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))

Epoch: 1 	Training Loss: 0.861093
Epoch: 2 	Training Loss: 0.588871
Epoch: 3 	Training Loss: 0.471986
Epoch: 4 	Training Loss: 0.447634
Epoch: 5 	Training Loss: 0.413171
Epoch: 6 	Training Loss: 0.426147
Epoch: 7 	Training Loss: 0.412496
Epoch: 8 	Training Loss: 0.391677
Epoch: 9 	Training Loss: 0.381983
Epoch: 10 	Training Loss: 0.350550


In [62]:
#check the mdodel on test data
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
    print(f'Accuracy: {100 * correct / total} %')

Accuracy: 62.5 %
